### Connect to Gremlin Server

In [1]:
import os
import sys
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)


from gremlin_python import statics
from gremlin_python.process.traversal import T, Direction
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import GraphTraversalSource
from gremlin_python.process.graph_traversal import __

from graph.base import g
from ipycytoscape_graph_visualization import visualize_graph

from dotenv import load_dotenv

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)

# test connection to gremlin server
g.V().limit(1).toList()

[v[53047360]]

### Analyze Address `1BBZ` and Export to .gexf

In [2]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
# interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)
# graph = analyzer.traversal_to_networkx(my_hist)

print(my_hist)
print(graph)

# dump graph to gexf file
addr_hist_graph_path = Path('/', 'app', 'addr_hist_graph.gexf')
nx.write_gexf(graph, addr_hist_graph_path)

coin_sources = analyzer.get_coin_traces(address.id, 'address', direction='incoming', graph=graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {round(source['amount'], 10)}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

id of address 1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs: 504
[['withStrategies', OptionsStrategy], ['withStrategies', OptionsStrategy]][['V'], ['has', 'address_id', 504], ['repeat', [['inE', 'sent'], ['otherV']]], ['emit'], ['path'], ['by', [['elementMap']]], ['by', [['elementMap']]], ['unfold']]
DiGraph with 10 nodes and 10 edges
amount from 187:1:0 15NU (1.0) is 1.0
amount from 183:1:0 13Ht (1.0) is 1.0
amount from 182:1:1 12cb (29.0) is 11.0
amount from 181:1:1 12cb (30.0) is 11.0
amount from 170:1:1 12cb (40.0) is 11.0
amount from 9:0:0 12cb (50.0) is 11.0
amount from 248:1:0 1ByL (10.0) is 10.0
amount from 183:1:1 12cb (28.0) is 10.0
amount from 360:0:0 18SH (50.0) is 50.0


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Analyze Address `1KAD`

In [3]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
# interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)

print(graph)

coin_sources = analyzer.get_coin_traces(address.id, 'address', direction='incoming', graph=graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {round(source['amount'], 10)}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

id of address 1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv: 557
DiGraph with 9 nodes and 13 edges
amount from 546:2:0 1KAD (1.0) is 0.96
amount from 546:1:0 1KAD (1.0) is 0.96
amount from 545:1:0 1DZT (1.0) is 0.96
amount from 524:1:0 1DCb (25.0) is 24.0
amount from 286:0:0 1Jhk (50.0) is 24.0
amount from 545:1:1 1DCb (24.0) is 23.04
amount from 546:1:1 1DZT (24.0) is 23.04
amount from 546:2:1 1KAD (24.0) is 23.04


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Trace Forwards from `1Jhk`

In [4]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

interesting_addr = '1Jhk2DHosaaZx1E4CbnTGcKM7FC88YHYv9'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_vertex_path(address.id, 'address')
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)

print(graph)

coin_sources = analyzer.get_coin_traces(address.id, 'address', direction='outgoing', graph=graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {round(source['amount'], 10)}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

id of address 1Jhk2DHosaaZx1E4CbnTGcKM7FC88YHYv9: 293
DiGraph with 11 nodes and 16 edges


KeyError: 52387992

### Analyzing an Output

In [ ]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

interesting_output = 561

my_hist = analyzer.get_output_history(interesting_output)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)

print(graph)

coin_sources = analyzer.get_coin_traces(interesting_output, 'output', graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {source['amount']}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

DiGraph with 3 nodes and 2 edges
Received 1.0 of 100000000 from 545:1:0 1DZT (1.0)
amount from 524:1:0 1DCb (25.0) is 1.0
amount from 286:0:0 1Jhk (50.0) is 1.0


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Traverse Forward

In [ ]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
# interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
# interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_path = analyzer.get_vertex_path(address.id, 'address')
graph = analyzer.traversal_to_networkx(my_path, include_data=True)

print(graph)

# print(analyzer.get_coin_destinations(address.id, 'address', graph, pretty_labels=True))

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

id of address 12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S: 9
DiGraph with 110 nodes and 115 edges


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…